In [1]:
import numpy as np
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
pd.set_option("display.precision", 1)

In [97]:
# Load and preprocess the data
courses = pd.read_excel('Rated_Courses.xlsx')
users = pd.read_excel('User_Profiles.xlsx')
ratings = pd.read_excel('User_Ratings.xlsx')
y_train = ratings.rating.values

## Cleaning

In [98]:
users = users.drop('Unnamed: 0',axis=1)
ratings = ratings.drop('Unnamed: 0',axis=1)
courses = courses.drop('Unnamed: 0',axis=1)
courses['id'] = courses['id'].astype(int)

## Feature Extraction

### User Data

In [99]:
# Merging courses into ratings and gettings dummies for subcategory.
df_merged = pd.merge(ratings,courses[['id','subcategory']], left_on='courseId', right_on='id').drop_duplicates()
df_subcategory = pd.get_dummies(df_merged['subcategory'])

# Contains each user rating with one hot encoding of course subcategory
df_ratings = pd.concat([ratings, df_subcategory], axis=1)

In [100]:
# Group by userid and category, and calculate the mean rating for each group
df_user_subcategory_rating = df_merged.groupby(["userId", "subcategory"])["rating"].mean().reset_index()

# Pivot the table to have category columns with average rating values
df_user_subcategory_rating = df_user_subcategory_rating.pivot(index="userId", columns="subcategory", values="rating").reset_index()

# Rename the columns for clarity
df_user_subcategory_rating.columns.name = None

# Display the new dataframe
df_user_average_ratings = df_user_subcategory_rating.replace(np.nan, 0)
user_input = df_user_average_ratings.iloc[:,1:]
user_input

,3D & Animation,Accounting & Bookkeeping,Affiliate Marketing,Apple,Architectural Design,Arts & Crafts,Beauty & Makeup,Branding,Business Analytics & Intelligence,Business Law,...,Teacher Training,Test Prep,Travel,User Experience Design,Video & Mobile Marketing,Video Design,Vocal,Web Design,Web Development,Yoga
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.3,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0
9996,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0
9998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0


### Courses Data

In [101]:
encoded_cols = pd.get_dummies(courses['subcategory'], prefix='')
encoded_cols = encoded_cols.rename(columns=lambda x: x.replace('_', ''))
df_courses = pd.concat([courses, encoded_cols], axis=1)

In [102]:
courses_input = df_courses.iloc[:, list([0,6]) + list(range(20, df_courses.shape[1]))]
courses_input.head()

,id,avg_rating,3D & Animation,Accounting & Bookkeeping,Affiliate Marketing,Apple,Architectural Design,Arts & Crafts,Beauty & Makeup,Branding,...,Teacher Training,Test Prep,Travel,User Experience Design,Video & Mobile Marketing,Video Design,Vocal,Web Design,Web Development,Yoga
0,638418,4.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,3640438,4.9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,4439592,4.5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,451966,4.3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1578238,3.9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
num_user_features = user_input.shape[1]  # remove userid, rating count and ave rating during training
num_item_features = courses_input.shape[1] -1 # remove movie id at train time
scaledata = True  # applies the standard scalar to data if true

In [104]:
# Group by userid and category, and calculate the mean rating for each group
# df_new2 = df_merged.groupby(["userId", "subcategory"])["rating"].mean().reset_index()

# Pivot the table to have category columns with average rating values


In [105]:
# use pivot to create a dataframe with subcategory as columns, rating as values, and userId as an additional column
df_pivot = df_merged.pivot( columns="subcategory", values="rating")
# df_pivot['userId'] = df_merged['userId']
df_pivot.insert(0, 'userId', df_merged['userId'])
# reset the index and flatten the column names
# df_pivot.columns = [' '.join(col).strip() for col in df_pivot.columns.values]
# df_pivot.drop_duplicates()
# df_pivot

In [106]:
cols_to_mean = df_pivot.columns[1:]
df_p = df_pivot.copy()
for col in cols_to_mean:
    df_p[f'{col}'] = df_p.groupby('userId')[col].transform('mean')
df_p = df_p.fillna(0)
user_input = df_p.fillna(0)

In [107]:
if scaledata:
    item_train_save = courses_input
    user_train_save = user_input

    scalerItem = StandardScaler()
    scalerItem.fit(courses_input)
    courses_input = scalerItem.transform(courses_input)

    scalerUser = StandardScaler()
    scalerUser.fit(user_input)
    user_input = scalerUser.transform(user_input)
    print(np.allclose(item_train_save, scalerItem.inverse_transform(courses_input)))
    print(np.allclose(user_train_save, scalerUser.inverse_transform(user_input)))

True
True


In [108]:
from sklearn.model_selection import train_test_split
item_train, item_test = train_test_split(courses_input, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_input, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test  data shape: {item_test.shape}")

movie/item training data shape: (27940, 132)
movie/item test  data shape: (6986, 132)


In [109]:
scaler = MinMaxScaler((-1, 1))
scaler.fit(y_train.reshape(-1, 1))
ynorm_train = scaler.transform(y_train.reshape(-1, 1))
ynorm_test = scaler.transform(y_test.reshape(-1, 1))
print(ynorm_train.shape, ynorm_test.shape)

(27940, 1) (6986, 1)


In [110]:
# GRADED_CELL
# UNQ_C1

num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###   
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs),
    ### END CODE HERE ###  
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 130)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 131)]        0           []                               
                                                                                                  
 sequential_2 (Sequential)      (None, 32)           70560       ['input_3[0][0]']                
                                                                                                  
 sequential_3 (Sequential)      (None, 32)           70816       ['input_4[0][0]']                
                                                                                            

In [111]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [112]:
tf.random.set_seed(1)
model.fit([user_train[:,1:], item_train[:,1:]], ynorm_train, epochs=100)

Epoch 1/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2155
Epoch 2/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2131
Epoch 3/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2128
Epoch 4/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2121
Epoch 5/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2115
Epoch 6/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2108
Epoch 7/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2103
Epoch 8/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2088
Epoch 9/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2077
Epoch 10/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2062
Epoch 11/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2046
Epoch 12/100
874/874 [==============================] - 2s 2ms/step - loss: 0.2033
Epoch 13/100


874/874 [==============================] - 2s 2ms/step - loss: 0.0970
Epoch 100/100
874/874 [==============================] - 2s 2ms/step - loss: 0.0967


In [114]:
model.evaluate([user_test[:, 1:], item_test[:, 1:]], ynorm_test)

219/219 [==============================] - 0s 1ms/step - loss: 0.3006


0.3005814552307129

In [115]:
def gen_user_vecs(user_vec, num_items):
    """ given a user vector return:
        user predict maxtrix to match the size of item_vecs """
    user_vecs = np.tile(user_vec, (num_items, 1))
    return (user_vecs)

In [116]:
new_user = [0]*131
new_user[0] = 10000
new_user[4] = 4.5
user_vec = np.array([new_user])
user_vecs = gen_user_vecs(user_vec,len(item_train))

In [117]:
def predict_uservec(user_vecs, item_vecs, model, scaler, ScalerUser, ScalerItem, scaledata=False):
    """ given a user vector, does the prediction on all movies in item_vecs returns
        an array predictions sorted by predicted rating,
        arrays of user and item, sorted by predicted rating sorting index
    """
    print(item_vecs)
    if scaledata:
        scaled_user_vecs = ScalerUser.transform(user_vecs)
        #scaled_item_vecs = ScalerItem.transform(item_vecs)
        y_p = model.predict(
            [scaled_user_vecs[:, 1:], item_vecs[:, 1:]])
    else:
        y_p = model.predict([user_vecs[:, 1:], item_vecs[:, 1:]])
    y_pu = scaler.inverse_transform(y_p)
    if np.any(y_pu < 0):
        print("Error, expected all positive predictions")
    # negate to get largest rating first
    sorted_index = np.argsort(-y_pu, axis=0).reshape(-1).tolist()
    sorted_ypu = y_pu[sorted_index]
    sorted_items = item_vecs[sorted_index]
    sorted_user = user_vecs[sorted_index]
    print(item_vecs)
    return y_pu
    return (sorted_index, sorted_ypu, sorted_items, sorted_user)

3836952.0

In [126]:
a = pd.DataFrame(scalerItem.inverse_transform(item_train), columns= list(['id','avg_rating'] + list(df_subcategory.columns)))
a['pred_rating'] = y_pu

In [128]:
a['id'] = a['id'].astype(int)

In [129]:
a

,id,avg_rating,3D & Animation,Accounting & Bookkeeping,Affiliate Marketing,Apple,Architectural Design,Arts & Crafts,Beauty & Makeup,Branding,...,Test Prep,Travel,User Experience Design,Video & Mobile Marketing,Video Design,Vocal,Web Design,Web Development,Yoga,pred_rating
0,3836952,3.9e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.3
1,3418022,3.8e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.2
2,4708652,4.4e-16,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.3
3,4732848,5.0e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.6
4,4125950,4.4e-16,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27935,2521354,4.1e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.6
27936,3920954,4.0e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.3
27937,3909484,3.8e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.6
27938,2401886,4.6e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.6


In [130]:
y_pu = predict_uservec(user_vecs, item_train, model, scaler, scalerUser, scalerItem, scaledata=scaledata)

[[ 6.20878276e-01  1.04394780e-01 -1.37276391e-01 ... -8.42224768e-02
  -1.61519316e-01 -8.85947050e-02]
 [ 3.12325878e-01  2.92158426e-02 -1.37276391e-01 ... -8.42224768e-02
  -1.61519316e-01 -8.85947050e-02]
 [ 1.26290708e+00 -2.41946886e+00 -1.37276391e-01 ... -8.42224768e-02
  -1.61519316e-01 -8.85947050e-02]
 ...
 [ 6.74299904e-01 -3.00369297e-03 -1.37276391e-01 ... -8.42224768e-02
  -1.61519316e-01 -8.85947050e-02]
 [-4.36083640e-01  5.16088659e-01 -1.37276391e-01 ... -8.42224768e-02
  -1.61519316e-01 -8.85947050e-02]
 [-5.98502122e-01  2.86972127e-01 -1.37276391e-01 ...  1.18733150e+01
  -1.61519316e-01 -8.85947050e-02]]
 45/874 [>.............................] - ETA: 0s  

C:\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


874/874 [==============================] - 1s 1ms/step
[[ 6.20878276e-01  1.04394780e-01 -1.37276391e-01 ... -8.42224768e-02
  -1.61519316e-01 -8.85947050e-02]
 [ 3.12325878e-01  2.92158426e-02 -1.37276391e-01 ... -8.42224768e-02
  -1.61519316e-01 -8.85947050e-02]
 [ 1.26290708e+00 -2.41946886e+00 -1.37276391e-01 ... -8.42224768e-02
  -1.61519316e-01 -8.85947050e-02]
 ...
 [ 6.74299904e-01 -3.00369297e-03 -1.37276391e-01 ... -8.42224768e-02
  -1.61519316e-01 -8.85947050e-02]
 [-4.36083640e-01  5.16088659e-01 -1.37276391e-01 ... -8.42224768e-02
  -1.61519316e-01 -8.85947050e-02]
 [-5.98502122e-01  2.86972127e-01 -1.37276391e-01 ...  1.18733150e+01
  -1.61519316e-01 -8.85947050e-02]]


In [48]:
scalerItem.inverse_transform(sorted_items)[0]

array([ 4.50000000e+00,  0.00000000e+00,  0.00000000e+00, -8.67361738e-19,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -8.67361738e-19,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  8.67361738e-19,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -8.67361738e-19,
        8.67361738e-19,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -3.46944695e-18,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [131]:
a.reindex(sorted_index)

,id,avg_rating,3D & Animation,Accounting & Bookkeeping,Affiliate Marketing,Apple,Architectural Design,Arts & Crafts,Beauty & Makeup,Branding,...,Test Prep,Travel,User Experience Design,Video & Mobile Marketing,Video Design,Vocal,Web Design,Web Development,Yoga,pred_rating
10726,4045364,4.5e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.9
233,3708820,3.5e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.9
15750,3038610,4.4e-16,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.9
20656,2683754,4.8e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.9
15731,861608,4.5e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,3870044,4.4e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.1
6458,3801808,3.9e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.1
5416,3101354,4.5e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.1
26353,1433316,4.5e+00,0.0,0.0,-8.7e-19,0.0,0.0,0.0,-8.7e-19,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5e-18,0.0,4.1


In [132]:
movie_dict = defaultdict(dict,courses.to_dict(orient='index')) 

In [135]:
def print_pred_movies(y_p, item, movie_dict, maxcount=10):
    """ print results of prediction of a new user. inputs are expected to be in
        sorted order, unscaled. """
    count = 0
    movies_listed = defaultdict(int)
    disp = [["y_p", "id", "title", "subcategory"]]
#     item = scalerItem.inverse_transform(item)
    print(item[0])
    for i in range(0, y_p.shape[0]):
        if count == maxcount:
            break
        count += 1
        movie_id = item[i, 0].astype(int)
        if movie_id in movies_listed:
            continue
        movies_listed[movie_id] = 1
        disp.append([y_p[i, 0], movie_id, item[i, 2].astype(float),
                    movie_dict[movie_id]['title'], movie_dict[movie_id]['subcategory']])
    table = tabulate.tabulate(disp, tablefmt='html', headers="firstrow")
    return (table)

In [136]:
print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

[ 0.48028934 -0.13727639 -0.09585733 -0.06611416 -0.03631539 -0.08283213
 -0.1633761  -0.08609856 -0.05304553 -0.07317143 -0.0397145  -0.06633224
 -0.10871946 -0.05772673 -0.0794353  -0.03510972 -0.07998098 -0.06633224
 -0.09646389 -0.08265674 -0.08542098 -0.06298321 -0.10667905 -0.12937913
 -0.10885419 -0.07777586 -0.03631539 -0.11386138 -0.11590815 -0.13856979
 -0.05249995 -0.10858457 -0.04939286 -0.05747655 -0.09416986 -0.09914922
 -0.04416754 -0.06826414 -0.06479055 -0.13198554 -0.07116704 -0.12949347
 -0.0397145  -0.0502591  -0.06479055 -0.06182724 -0.06017217 -0.05944895
 -0.159171   -0.04939286 -0.04182829 -0.18100723 -0.04416754 -0.16964216
 -0.1524226  -0.06545564 -0.09855856 -0.04939286 -0.07589054 -0.07569948
 -0.08353012 -0.03898462 -0.0695228  -0.0482139  -0.09751664 -0.23925367
 -0.07925259 -0.04384095 -0.04217035 -0.0963126  -0.11463297 -0.07758934
 -0.07777586 -0.11256423 -0.04148342  9.57297811 -0.07297342 -0.08106161
 -0.06135885 -0.04351192 -0.05671945 -0.07888593 -0

KeyError: 'title'